<a href="https://colab.research.google.com/github/melancholymans/Bonanza/blob/master/TalkingData_AdTracking_Fraud_Detection_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TalkingData_AdTracking_Fraud_Detection_Challenge

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
A = np.array([[-0.259255,0.637216],
                     [-0.163380,0.676041],
                     [0.681100,0.122803],
                     [0.483628,0.075390],
                     [0.456400,0.340823]])
B = np.array([[2.400068,0.0],
             [0.0,3.127071]])
C = np.array([[-0.054005,-0.284113,0.675451,0.473945,0.485289],
            [0.745145,0.623738,0.172371,0.148262,0.063380]])

In [3]:
Z=A@B@C
print(A.shape)
print(B.shape)
print(C.shape)
print(Z.shape)
df = pd.DataFrame(Z)

df

(5, 2)
(2, 2)
(2, 5)
(5, 5)


,0,1,2,3,4
0,1.518394,1.419656,-0.076816,0.000527,-0.175669
1,1.596434,1.430007,0.099537,0.127585,-0.056306
2,0.197865,-0.224912,1.170343,0.831686,0.817634
3,0.112982,-0.182735,0.824659,0.585080,0.578236
4,0.735002,0.353551,0.923592,0.677169,0.599130


https://www.rco.recruit.co.jp/career/engineer/blog/kaggle_talkingdata_basic/

## Kaggle TalkingData Fraud Detection コンペの解法まとめ(基本編)

こんにちは。今年2018年4月より新卒でRCOに入社した松田です。

kaggle というデータ分析のコンペティション運営サイトが流行っていますが、 RCOでも「kaggle部」という形で積極的に参加しています。

今回私は TalkingData AdTracking Fraud Detection Challenge というコンペティションに参加したので、そこで得た知見を共有したいと思います。


- カラム名	意味	種類	例(筆者の想像)
- ip	ユーザーのIPアドレス	110116	192.168.0.1
- app	広告が宣伝するモバイルアプリ	348	パズルクエスト
- device	ユーザーの端末	749	iPhone7
- os	ユーザーのOS (versionも区別)	244	iOS 11.3.1
- channel	広告媒体のチャネル	177	ラッキーニュース
- click_time	広告がクリックされた時刻(UTC)	-	-


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
"""

train.csvファイルがでかくてread_csv関数では呼び込めないのでawkでis_attributedが1であるファイルを抽出したプログラム(clearn.awk)を作った
同じくヘッダーがわからなかったので３行未満だけ読み出すhead.awkを作った

-head.awk
# >awk -f head.awk train.csv 
BEGIN{
    FS = ","
}
NR < 3 {
    print $0
}


- clearn.awk
# >awk -f head.awk train.csv 
# ip,app,device,os,channel,click_time,attributed_time,is_attributed
BEGIN {
    FS = ","
    count = 0
}
$8 == "0"{
    print $0;
    if (count > 460000) exit;
    count = count + 1;
}
"""


'\n\ntrain.csvファイルがでかくてread_csv関数では呼び込めないのでawkでis_attributedが1であるファイルを抽出したプログラム(clearn.awk)を作った\n同じくヘッダーがわからなかったので３行未満だけ読み出すhead.awkを作った\n\n-head.awk\n# >awk -f head.awk train.csv \nBEGIN{\n    FS = ","\n}\nNR < 3 {\n    print $0\n}\n\n\n- clearn.awk\n# >awk -f head.awk train.csv \n# ip,app,device,os,channel,click_time,attributed_time,is_attributed\nBEGIN {\n    FS = ","\n    count = 0\n}\n$8 == "0"{\n    print $0;\n    if (count > 460000) exit;\n    count = count + 1;\n}\n'

In [0]:
base_path = 'drive/My Drive/Colab Notebooks/kaggle/TalkingData_AdTracking_Fraud_Detection_Challenge/input/'

In [7]:
df = pd.read_csv(base_path+'train.csv',encoding='utf-16')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
1,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1
2,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1
3,172429,35,1,46,274,2017-11-06 16:00:03,2017-11-07 00:55:29,1
4,199085,35,1,13,274,2017-11-06 16:00:04,2017-11-06 23:04:54,1


In [9]:
aggregation = {'app':{'ip_count' : 'count'}}

df1= df.groupby(['ip']).agg(aggregation).reset_index()

/usr/local/lib/python3.6/dist-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [10]:
df1.head()

,ip,app
,,ip_count
0,1,9
1,6,2
2,9,8
3,10,4
4,19,4


In [11]:
df.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
1,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1
2,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1
3,172429,35,1,46,274,2017-11-06 16:00:03,2017-11-07 00:55:29,1
4,199085,35,1,13,274,2017-11-06 16:00:04,2017-11-06 23:04:54,1


In [12]:
# test_dfとagg_dataのstoreid_id、dowをすり合わせmergeさせる
merged = pd.merge(df, df1, how='left', left_on=['ip'], right_on=['ip'])

/usr/local/lib/python3.6/dist-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [13]:
merged.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,"(app, ip_count)"
0,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,1
1,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1,11
2,64516,35,1,13,21,2017-11-06 16:00:02,2017-11-06 23:40:50,1,88
3,172429,35,1,46,274,2017-11-06 16:00:03,2017-11-07 00:55:29,1,2
4,199085,35,1,13,274,2017-11-06 16:00:04,2017-11-06 23:04:54,1,4


In [14]:
merged[merged['ip']==29692]

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,"(app, ip_count)"
1,29692,9,1,22,215,2017-11-06 16:00:02,2017-11-07 10:05:22,1,11
148584,29692,19,6,29,213,2017-11-07 15:02:06,2017-11-07 15:03:09,1,11
154679,29692,84,0,24,101,2017-11-07 16:37:07,2017-11-08 04:53:50,1,11
342785,29692,27,1,47,122,2017-11-09 02:33:53,2017-11-09 15:07:47,1,11
531920,29692,12,1,22,178,2017-11-06 16:01:20,NaN,0,11
594116,29692,3,1,19,280,2017-11-06 16:02:32,NaN,0,11
594145,29692,12,1,19,178,2017-11-06 16:02:32,NaN,0,11
594168,29692,9,1,19,215,2017-11-06 16:02:32,NaN,0,11
594174,29692,9,1,19,134,2017-11-06 16:02:32,NaN,0,11
595965,29692,3,1,19,137,2017-11-06 16:02:34,NaN,0,11
